# Connecting to Azure ML Client

In [75]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration, Environment, OnlineRequestSettings
from azure.identity import DefaultAzureCredential
import os

# Initialize MLClient
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)


Found the config file in: .\config.json


# Define the endpoint and create it

In [ ]:
# Define the endpoint
endpoint_name = "medical-question-endpoint"
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint for OpenAI and you.com API based inference for answering medical questions",
    auth_mode="key"
)

# Create the endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# Creating an environment with the required dependencies

In [31]:
# Define the environment
medical_question_env = Environment(
    name="medical-question-env",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="medical_question_env.yml",
    description="Environment created from a Docker image plus Conda environment that includes the necessary dependencies for the medical question answering model"
)
# mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04

# Create the environment
ml_client.environments.create_or_update(medical_question_env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'medical-question-env', 'description': 'Environment created from a Docker image plus Conda environment that includes the necessary dependencies for the medical question answering model', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/074fef04-0bdf-4f33-8f67-8609f7d113dd/resourceGroups/Marcel_Students/providers/Microsoft.MachineLearningServices/workspaces/Marcel_PlaygroundML/environments/medical-question-env/versions/20', 'Resource__source_path': '', 'base_path': 'c:\\Users\\marce\\Documents\\2 Bcas-Qualentum\\Trabajo Final de Bootcamp\\LiveQA_MedicalTask_TREC2017\\azure-ml-deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x00000259AB6DB79

# Deploying the model/python script

In [93]:
# Define the deployment
endpoint_name = "medical-question-endpoint"
deployment = ManagedOnlineDeployment(
    name="medical-question-deployment",
    endpoint_name=endpoint_name,
    environment=medical_question_env,
    code_configuration=CodeConfiguration(code="code/", scoring_script="azure_inference_script.py"),
    request_settings=OnlineRequestSettings(request_timeout_ms=20000),
    instance_type="Standard_F2s_v2",
    instance_count=1,
)

# Create the deployment
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint medical-question-endpoint exists
Uploading code (0.01 MBs): 100%|##########| 7357/7357 [00:00<00:00, 143881.93it/s]




...........................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'medical-question-endpoint', 'type': 'Managed', 'name': 'medical-question-deployment', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/074fef04-0bdf-4f33-8f67-8609f7d113dd/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/odidp:b67618ed-6265-41ce-a880-14ca3d377412:84eda217-5cf2-4f9b-866b-0007b5dd2b7d?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/074fef04-0bdf-4f33-8f67-8609f7d113dd/resourceGroups/Marcel_Students/providers/Microsoft.MachineLearningServices/workspaces/Marcel_PlaygroundML/onlineEndpoints/medical-question-endpoint/deployments/medical-question-deployment', 'Resource__source_path': '', 'base_path': 'c:\\Users\\marce\\Documents\\2 Bcas-Qualentum\\Trabajo Final de Bootcamp\\LiveQA_MedicalTask_TREC2017\\azu

# Activate the deployment by sending 100% of the traffic to it

In [94]:
# Set the traffic to 100% for the deployment
endpoint.traffic = {"medical-question-deployment": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint_name} is up and running.")

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


Endpoint medical-question-endpoint is up and running.


# Verifying the listed endpoints

In [95]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

# Get the details for online endpoint
endpoint_to_check = ml_client.online_endpoints.get(name="medical-question-endpoint")

# existing traffic details
print(endpoint_to_check.traffic)

# Get the scoring URI
print(endpoint_to_check.scoring_uri)

endpoint = ml_client.online_endpoints.get(name=endpoint_name)

endpoint-08051858371678
medical-question-endpoint
{'medical-question-deployment': 100}
https://medical-question-endpoint.westeurope.inference.ml.azure.com/score


# Testing the deployment

In [96]:
import json

# Creating an example question in an appropriate JSON format
test_question = "My daughter has menkes syndrome and would like to know how to go about. Thx"
test_question_json = {"question": test_question}

# Save the question to a file in proper JSON format
with open("test_question.json", "w") as f:
    json.dump(test_question_json, f)

# Save the name of the file in a variable
test_question_file = "test_question.json"

In [98]:
# Testing the medical-question-deployment with some sample question
endpoint_name = "medical-question-endpoint"
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="medical-question-deployment",
    request_file="test_question.json",
)

In [105]:
response

'"{\\"response\\": \\"Category of question: Information\\\\nSearch Query and focus of the question: management of Menkes syndrome\\\\nAnswer: Menkes syndrome, also known as Menkes disease, is a genetic disorder that affects copper levels in the body, leading to various health challenges. Here are some steps you can take to manage the condition:\\\\n\\\\n1. **Consult Healthcare Professionals**: Regular visits to a pediatrician, neurologist, and geneticist are essential. They can help monitor your daughter\'s development and manage symptoms.\\\\n\\\\n2. **Copper Supplementation**: Early treatment with copper injections may improve outcomes if started shortly after birth. Discuss with your healthcare provider about the possibility of copper supplementation and the best approach for your daughter.\\\\n\\\\n3. **Nutritional Support**: Ensure that your daughter has a balanced diet that supports her overall health. Consult a nutritionist familiar with Menkes syndrome for tailored dietary advi

In [106]:
# Step 1: Load the initial JSON string
decoded_response = json.loads(response)

# Step 2: Load the decoded JSON string to access the actual response data
response_data = json.loads(decoded_response)
response_text = response_data['response']

print(response_text)

Category of question: Information
Search Query and focus of the question: management of Menkes syndrome
Answer: Menkes syndrome, also known as Menkes disease, is a genetic disorder that affects copper levels in the body, leading to various health challenges. Here are some steps you can take to manage the condition:

1. **Consult Healthcare Professionals**: Regular visits to a pediatrician, neurologist, and geneticist are essential. They can help monitor your daughter's development and manage symptoms.

2. **Copper Supplementation**: Early treatment with copper injections may improve outcomes if started shortly after birth. Discuss with your healthcare provider about the possibility of copper supplementation and the best approach for your daughter.

3. **Nutritional Support**: Ensure that your daughter has a balanced diet that supports her overall health. Consult a nutritionist familiar with Menkes syndrome for tailored dietary advice.

4. **Physical and Occupational Therapy**: These th